In [ ]:
import warnings
import datetime as dt
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['W1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    Src['Datetime']                     = Query['datetime']
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Variat']       = (Src['Close'] / Src['Open']) *100-100
    Src['Hilo']         = (Src['High']  / Src['Low'])  *100-100

    Src['Sign Bull']    = Src['Variat'].apply(lambda x: +1 if x >= 0 else nan)
    Src['Sign Bear']    = Src['Variat'].apply(lambda x: -1 if x <  0 else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    # Calc['Var Abs'] = Calc['Variat'].abs()
    # Calc['Var Pos'] = Calc['Variat'] * Calc['Sign Bull']
    # Calc['Var Neg'] = Calc['Variat'] * Calc['Sign Bear']
    # Calc['HL Pos']  = Calc['Hilo']   * Calc['Sign Bull']
    # Calc['HL Neg']  = Calc['Hilo']   * Calc['Sign Bear']


    FUTS = [0, 1, 2, 3, 4]
    for x in FUTS:  Calc[f'Variat +{x}']  = Calc['Variat']  .shift(-x)
    for x in FUTS:  Calc[f'Hilo +{x}']    = Calc['Hilo']    .shift(-x)
    # for x in FUTS:  Calc[f'Var Abs +{x}'] = Calc['Var Abs'] .shift(-x)
    # for x in FUTS:  Calc[f'Var Pos +{x}'] = Calc['Var Pos'] .shift(-x)
    # for x in FUTS:  Calc[f'Var Neg +{x}'] = Calc['Var Neg'] .shift(-x)
    # for x in FUTS:  Calc[f'HL Pos +{x}']  = Calc['HL Pos']  .shift(-x)
    # for x in FUTS:  Calc[f'HL Neg +{x}']  = Calc['HL Neg']  .shift(-x)

    
    for x in FUTS:  Calc[f'Variat +{x} Pos']  = (Calc[f'Variat +{x}'] >= 0)
    for x in FUTS:  Calc[f'Variat +{x} Neg']  = (Calc[f'Variat +{x}'] <  0)
        

    for win in pd.Series(FUTS).expanding():
        A, Z = win.iloc[0], win.iloc[-1]
        
        Calc[f'Variat {A}-{Z} Pos'] = Calc.loc[:, f'Variat +{A} Pos' : f'Variat +{Z} Pos'].all(axis=1)
        Calc[f'Variat {A}-{Z} Neg'] = Calc.loc[:, f'Variat +{A} Neg' : f'Variat +{Z} Neg'].all(axis=1)

    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [12]:
def STATS(Calc, FUTS=[0, 1, 2, 3, 4]): 

    pipe = []
    for win in [win for win in pd.Series(FUTS).expanding() if len(win) >= 2]:
        A = win.iloc[ 0]
        B = win.iloc[-2]
        C = win.iloc[-1]

        Df1 = Calc[Calc[f'Variat {A}-{B} Pos']] 
        pipe.append({
            'Cond':                       f'Variat {A}-{B} Pos',
            'Variat +1 Pos': np.round(Df1[f'Variat +{C} Pos'] .mean()*100, 1), 
            'Variat +1 Neg': np.round(Df1[f'Variat +{C} Neg'] .mean()*100, 1), 
        })

        Df2 = Calc[Calc[f'Variat {A}-{B} Neg']] 
        pipe.append({
            'Cond':                       f'Variat {A}-{B} Neg',
            'Variat +1 Pos': np.round(Df2[f'Variat +{C} Pos'] .mean()*100, 1), 
            'Variat +1 Neg': np.round(Df2[f'Variat +{C} Neg'] .mean()*100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

STATS(Calc)

,Cond,Variat +1 Pos,Variat +1 Neg
0,Variat 0-0 Pos,53.1,46.9
1,Variat 0-0 Neg,45.1,54.1
2,Variat 0-1 Pos,61.8,38.2
3,Variat 0-1 Neg,44.4,54.2
4,Variat 0-2 Pos,52.4,47.6
5,Variat 0-2 Neg,59.0,41.0
6,Variat 0-3 Pos,63.6,36.4
7,Variat 0-3 Neg,62.5,37.5


# Snippets

In [13]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)

    FUTS = [0, 1, 2, 3, 4]
    for x in FUTS:  Calc[f'Variat +{x}']  = Calc['Variat']  .shift(-x)
    for x in FUTS:  Calc[f'Hilo +{x}']    = Calc['Hilo']    .shift(-x)
    
    for x in FUTS:  Calc[f'Variat +{x} Pos']  = (Calc[f'Variat +{x}'] >= 0)
    for x in FUTS:  Calc[f'Variat +{x} Neg']  = (Calc[f'Variat +{x}'] <  0)
        
    for win in pd.Series(FUTS).expanding():
        A, Z = win.iloc[0], win.iloc[-1]
        Calc[f'Variat {A}-{Z} Pos'] = Calc.loc[:, f'Variat +{A} Pos' : f'Variat +{Z} Pos'].all(axis=1)
        Calc[f'Variat {A}-{Z} Neg'] = Calc.loc[:, f'Variat +{A} Neg' : f'Variat +{Z} Neg'].all(axis=1)

    return Calc


def STATS(Calc, FUTS=[0, 1, 2, 3, 4]): 
    pipe = []
    for win in [win for win in pd.Series(FUTS).expanding() if len(win) >= 2]:
        A = win.iloc[ 0]
        B = win.iloc[-2]
        C = win.iloc[-1]

        Df1 = Calc[Calc[f'Variat {A}-{B} Pos']] 
        pipe.append({
            'Cond':                       f'Variat {A}-{B} Pos',
            'Variat +1 Pos': np.round(Df1[f'Variat +{C} Pos'] .mean()*100, 1), 
            'Variat +1 Neg': np.round(Df1[f'Variat +{C} Neg'] .mean()*100, 1), 
        })

        Df2 = Calc[Calc[f'Variat {A}-{B} Neg']] 
        pipe.append({
            'Cond':                       f'Variat {A}-{B} Neg',
            'Variat +1 Pos': np.round(Df2[f'Variat +{C} Pos'] .mean()*100, 1), 
            'Variat +1 Neg': np.round(Df2[f'Variat +{C} Neg'] .mean()*100, 1), 
        })
    pass
    return pd.DataFrame(pipe)